# LowEmission Oil and Gas Open (LEOGO) reference platform
# October 2021
Representing a hypothetical, but realistic Norwegian offshore oil and gas platform

* wellstream: 50 Sm3/s, of which:
    * gas: 49.75 Sm3/s = 4.3 mill Sm3/day = 4300 Sm3oe/day
    * oil:  0.1 Sm3/s = 8640 Sm3/day  
    * water: 0.15 Sm3/s = 12960 Sm3/day 

Gas oil ratio and water cut (standard units):
* GOR = 4.3 mill/8640 = 497
* WC = 12960/(12960+8640) = 0.6


In [73]:
%load_ext autoreload
%autoreload 2

from oogeso.io import file_io
import oogeso.plots as plots
from oogeso.utils import create_time_series_data
import oogeso.dto
import matplotlib.pyplot as plt
import IPython.display
import pandas as pd
import logging
import pprint
import ipywidgets
import pickle
import plotly.express as px
import pyomo.environ as pyo
from dataclasses import asdict

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel('INFO') #INFO
#print("Using Oogeso version {}".format(oogeso.__version__))

[autoreload of numpy.matrixlib failed: Traceback (most recent call last):
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/Cellar/python@3.9/3.9.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/Cellar/python@3.9/3.9.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/morten.aslesen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of attr.validators failed: Traceback (most recent call last):
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/Users/morten.aslesen/Projects/oogeso/venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignment: '_OptionalValidator' object layout differ

## Read input data

In [74]:
case='Base' # base case without wind
#case='A' # case with 24 MW wind capacity
#case='B' # case with 24 MW wind capacity AND 4 MW, 4 MWh battery
#case="C"
#case="all"
#timerange=[0,12*24*7] #one week (with 5 min timestep)
timerange=[0,200] # testing
#timerange=[0,4]
#timerange=[1030,1050] #no wind period
outpath = "result/"
plots.plotter="plotly"

data = file_io.read_data_from_yaml('leogo_reference_platform.yaml')
timeseries = file_io.read_profiles_from_csv(
    filename_nowcasts="leogo_profiles_nowcasts.csv",
    filename_forecasts="leogo_profiles_forecasts.csv")
profiles = create_time_series_data(
    df_forecast= timeseries["forecast"],
    df_nowcast=timeseries["nowcast"],
    time_start=None,time_end=None,
    timestep_minutes=5,
    resample_method="linear")
data.profiles = profiles

#data['paramParameters']['planning_horizon']=12

incl_battery=0
incl_hydrogen=0
if case=='Base':
    wind_P_max=0 # no wind
elif case=='A':
    wind_P_max=8*3 # 3x8 MW, as agreed with ABB
elif case=='B':
    wind_P_max=8*3 # 3x8 MW, as agreed with ABB
    incl_battery=1
elif case=="C":
    wind_P_max=8*12 # Large wind farm Pmax=2x demand
    incl_hydrogen=1
    data.parameters.co2_tax=1000
    prof_h2target=oogeso.dto.oogeso_input_data_objects.TimeSeriesData(id="h2target",data=[0.5]*len(data.profiles[0].data))
    data.profiles.append(prof_h2target)
elif case=='BaseX':
    # CASE for illustration (to make plots for paper)
    wind_P_max=0 # no wind
    # oilgas prod down at timestep 100
    xlist=data.profiles[2].data
    pr2=[xlist[i]*0.8 if (i>100) and (i<150) else xlist[i] for i in range(len(xlist))]
    data.profiles[2].data=pr2
elif case=="all":
    # CASE for illustration (to make plots for paper)
    wind_P_max=8*12 # Large wind farm Pmax=2x demand
    incl_battery=1
    incl_hydrogen=1
    data.parameters.co2_tax=1000
    prof_h2target=oogeso.dto.oogeso_input_data_objects.TimeSeriesData(id="h2target",data=[0.5]*len(data.profiles[0].data))
    data.profiles.append(prof_h2target)


gts = ['Gen1','Gen2','Gen3']
wells = ['wellL1', 'wellL2']
hydrogendevices = ['electrolyser','fuelcell','h2storage']
for dev in data.devices:
    if dev.id in gts:
        dev.start_stop.is_on_init = True
    if dev.id == "wind":
        dev.flow_max = wind_P_max
    if dev.id == "battery":
        dev.include = incl_battery
    if dev.id in wells:
        # fixed production - not allowed to reduce:
        dev.flow_min = dev.flow_max
    if dev.id in hydrogendevices:
        dev.include = incl_hydrogen

simulator = oogeso.Simulator(data)


FileNotFoundError: [Errno 2] No such file or directory: 'leogo_reference_platform.yaml'

In [ ]:
# Test solve:
# simulator.optimiser.solve(solver="gurobi")

In [ ]:
#simulator.optimiser.paramDeviceEnergyInitially.pprint()

In [ ]:
#simulator.optimiser.dual.pprint()

In [ ]:
#optimiser.paramProfiles.pprint() # all 1 before update, from profile after
#optimiser.paramDeviceIsOnInitially.pprint() # all 1 before and after update
#optimiser.paramDevicePrepTimestepsInitially.pprint() # all 0 before and after update
#optimiser.paramDevicePowerInitially.pprint() # unchanged before/after

### Inspect input data

In [ ]:
@ipywidgets.interact(datagroup=['','devices','nodes','edges','carriers','parameters'])
def showdata(datagroup):
    pprint.pprint(asdict(data)[datagroup],indent=1) if datagroup!='' else print('')

In [ ]:
yy=['']+list(simulator.optimiser.component_objects(oogeso.optimiser.pyo.Constraint, active=True))
@ipywidgets.interact(constraint=yy)
def showdata(constraint):
    pprint.pprint(constraint.pprint(),width=1) if constraint!='' else print('')

In [ ]:
#data.devices[25]
#for e in data.edges: print(e.id,e.bidirectional)

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize=(8,3))
ax=plt.gca()
#day 6, hour 22: 6*24*12 + 22*12 = 
#tstart=7*24*12
#tlen=12*12
#xvalues = pd.date_range(start="00:00:00",freq="5min",periods=tlen)
#myFmt = mdates.DateFormatter('%H:%M')
#ax.xaxis.set_major_formatter(myFmt)
tstart=0
tlen=7*24*12
xvalues=range(tstart,tstart+tlen)
print(tstart,tlen)
for pr in profiles:
    if pr.id in ["curve_const","waterinj","h2target"]: continue
    color = next(ax._get_lines.prop_cycler)['color']
    plt.step(xvalues,pr.data[tstart:(tstart+tlen)],label=pr.id,color=color,where="post")
    if pr.data_nowcast is not None:
        color = next(ax._get_lines.prop_cycler)['color']
        plt.step(xvalues,pr.data_nowcast[tstart:(tstart+tlen)],"--",
                 label="{} (nowcast)".format(pr.id),color=color,where="post")        
plt.legend(loc="upper left",bbox_to_anchor=(1,1),frameon=False)
plt.xlabel("Time")
plt.ylabel("Relative value")
plt.savefig(outpath+"timerseries_ts{}_{}.pdf".format(tstart,case),dpi=300,bbox_inches="tight")

In [ ]:
outpath

In [ ]:
# View profiles (entire available time windows)
scale=300/96 # ratio print vs screen dpi
fig=plots.plot_profiles(profiles,filename=None)
fig.update_layout(template="plotly_white",autosize=False,width=600,height=200,margin=dict(l=0,r=0,t=0,b=0))
fig.show()
#300 dpi and 4 inch: 1200 pixels
fig.write_image(outpath+"profiles.pdf",width=600,height=200,scale=scale)

In [ ]:
dev=simulator.optimiser.all_devices['Gen1'].dev_data
fuelA = dev.fuel_A
fuelB = dev.fuel_B
Pmax = dev.flow_max
#plots.plotGasTurbineEfficiency(filename='gasturbine_Gen1.png',fuelA=fuelA,fuelB=fuelB,Pmax=Pmax)

In [ ]:
# View energy system
dotG=plots.plot_network(simulator,timestep=None,filename=None,rankdir="LR",hide_edgelabel=True,only_carrier=None,fontsize=20,fontname = "helvetica",)
IPython.display.Image(dotG.create_png(prog="dot")) #scales automatically to page width
#dotG.write_pdf(path=outpath+"diagram0_{}.pdf".format(case))

### Some checks on input data

In [ ]:
print("flow per pipe = {:.4} Sm3/s".format(50/15))
flow1=oogeso.networks.pipelines.darcy_weissbach_Q(p1=6.164,p2=6,f=0.001,rho=5.63,
    diameter_mm=200,length_km=1,height_difference_m=100)
flow2=oogeso.networks.pipelines.darcy_weissbach_Q(p1=6.164,p2=4,f=0.0136,rho=5.63,
    diameter_mm=200,length_km=1,height_difference_m=100)
flow3=oogeso.networks.pipelines.darcy_weissbach_Q(p1=7,p2=4,f=0.0189,rho=5.63,
    diameter_mm=200,length_km=1,height_difference_m=100)
print("flow1 (6.164->6): {:.4}".format(flow1))
print("flow2 (6.164->4): {:.4}".format(flow2))
print("flow3 (7->4):     {:.4}".format(flow3))

In [ ]:
oogeso.networks.pipelines.darcy_weissbach_p2(p1=25e6,Q=2.37,f=0.001,rho=5.63,
    diameter=0.2,length=1000,height_difference=100,p0_from=25e6,p0_to=24e6,linear=False)

In [ ]:
# Water pipe pressure drop ('w3')
#mc.instance.paramEdge['w3']['num_pipes']=15
#mc.instance.paramEdge['w1']['diameter_mm']=200
edge = simulator.optimiser.all_edges['w3']
#edge.edge_data.pressure_from=25.12
water_network = simulator.optimiser.all_networks["water"]
#water_network.carrier_data.viscosity=None
#water_network.carrier_data.darcy_friction=0.01
print(water_network.carrier_data)
print(edge.edge_data)
print("=> Pressure out={:.4} MPa".format(water_network.compute_edge_pressuredrop(edge,
    p1=25.063,Q=0.277/15,linear=False)))
print("Nonlinear computation of Q from pressure difference: Q={:.3g} Sm3/s".format(
    15*oogeso.networks.pipelines.darcy_weissbach_Q(p1=25.063,p2=26,f=0.01,rho=1000,diameter_mm=135,length_km=1,height_difference_m=-100)))

In [ ]:
# Darcy -Weissbach flow calculations - testing
import numpy as np
p10=25.063e6 # Pa
p20=26e6 # Pa
D=0.135 # m
f=0.01
L=1100 # m
z=-100 # m
rho=1000
grav=9.8
p1=25.081e6 # Pa
Q=0.277
num_pipes=15
k=np.sqrt(np.pi**2*D**5/(8*rho*f*L))
X=np.sqrt((p10-p20)-rho*grav*z)
p2 = p1 -(Q/num_pipes-k*X)*2*X/k -(p10-p20)
print("k={}".format(k))
print("X={}".format(X))
print("qhat={:.4g} Sm3/s (per pipe), {:.4g} Sm3/s (total)".format(X*k,15*k*X))
print("p2={:.6g} MPa (linear computation)".format(p2*1e-6))
print("flow part: {:.4g}, static part: p20-p10: {}".format(-1e-6*(Q/num_pipes-k*X)*2*X/k,(p20-p10)))
print("--")
k2 = (8 * f * rho * L) / (np.pi ** 2 * D ** 5)
q0 = np.sqrt(1 / k2 * ((p10 - p20) - rho * grav * z))
p22 = p1 - rho * grav * z - k2 * (2 * q0 * Q/num_pipes - q0 ** 2)
print("k2 = {}  (linear)".format(k2))
print("q0 = {:.4g} Sm3/s ({:.4g} Sm3/s total)".format(q0,q0*num_pipes))
print("p2 = {:.9g} MPa (linear)".format(p22*1e-6))
print("Pressure change: static={:.4g} MPa, friction={:.4g} MPa".format(
    -rho*grav*z*1e-6,-k2*(2*q0*Q/num_pipes-q0**2)*1e-6))
assert np.abs(p22-25.9960035e6)<1, "Pressure calculation error"
print("--")
(p222,q000)=oogeso.networks.pipelines.darcy_weissbach_p2(
    Q/num_pipes,p1=p1,f=f,rho=rho,diameter=D,length=L,height_difference=z,linear=True,p0_from=p10,p0_to=p20)
print("q0= {} Sm3/s (oogeso)".format(q000))
print("p2= {} MPa (oogeso)".format(p222*1e-6))
(p222,q000)=oogeso.networks.pipelines.darcy_weissbach_p2(
    Q/num_pipes,p1=p1,f=f,rho=rho,diameter=D,length=L,height_difference=z,linear=False,p0_from=p10,p0_to=p20)
print("p2= {} MPa (oogeso, nonlinear)".format(p222*1e-6))


In [ ]:
25.12, 25.063, 25.081

In [ ]:
print(simulator.optimiser.all_edges['w3'].edge_data)

## Solve

If the problem is infeasible, try 
* relaxing pressure deviation limits (generic value and per-node value)
* add a high cost emergency generator (that could represent load shedding) to ensure energy balance can be satisfied
* make sure excess heat/water/gas has a place to go


In [ ]:
dev_d=simulator.optimiser.all_devices["Gen1"].dev_data
node_d=simulator.optimiser.all_nodes["powersupply"].node_data

In [ ]:
#simulator.optimiser.write("testplatform.mps")
sim_result = simulator.runSimulation(solver="gurobi",timerange=timerange,write_yaml=False,timelimit=20)
print("Mean CO2 emission rate      = {:.1f} kgCO2/s".format(sim_result.dfCO2rate.mean()))
print("Mean CO2 emission intensity = {:.1f} kgCO2/Sm3oe".format(sim_result.dfCO2intensity.mean()))
print("Mean export revenue         =",*["{}:{:.1f} ".format(x,v) for x,v in sim_result.dfExportRevenue.mean().items() if v!=0],"$/s")

In [ ]:
#simulator.optimiser.updateOptimisationModel(timestep=0,profiles=data.profiles,first=True)
simulator.optimiser.write("test.mps",io_options = {"symbolic_solver_labels":True})

In [ ]:
#simulator.optimiser.constr_injectionwell_Pmax.pprint()
#simulator.optimiser.constrO_elReserveMargin[6].pprint()
pyo.value(simulator.optimiser.varEdgeFlow["w2",0])
#simulator.optimiser.constrE_w2_flow.pprint()

In [ ]:
print(pyo.value(simulator.optimiser.paramProfiles["curve_wind",6]),
    pyo.value(simulator.optimiser.varDeviceIsOn["Gen1",6]),
    pyo.value(simulator.optimiser.varDeviceIsOn["Gen2",6]),
    pyo.value(simulator.optimiser.varDeviceIsOn["Gen3",6]),
)

## Save/Load simulation results
By saving to pickle file, it easy to later open and analyse the results without having to re-run the simulation

In [ ]:
pickle_save='{}case{}.pkl'.format(outpath,case)
# Save (pickle) - for later opening and analysis
dumpdata={"data":data,"result":sim_result}
with open(pickle_save, mode='wb') as file:
   pickle.dump(dumpdata, file)
print("Results were saved to {}".format(pickle_save))

In [ ]:
# Open previously saved object (including simulation results)
#with open(pickle_save, mode='rb') as file:
#   sim_data = cloudpickle.load(file)
#data=sim_data["data"]
#sim_result=sim_data["result"]

## Analyse results

In [ ]:
import oogeso.result_analysis
kpi = oogeso.result_analysis.compute_kpis(sim_result, sim_data=data, wind_turbines=['wind'])
kpi = pd.DataFrame.from_dict(kpi,orient="index",columns=[case])
kpi 

In [ ]:
tstep=151
gCombined = plots.plot_network(simulator=simulator,timestep=tstep,numberformat="{:.3g}",
                              filename=None,only_carrier=None,fontsize=30,fontname = "helvetica",
                              hide_losses=True)
IPython.display.Image(gCombined.create_png())
#gCombined.write_png(path=outpath+"diagram_{}.pdf".format(case),f="pdf")
#gCombined.write_pdf(path=outpath+"diagram_{}.pdf".format(case))

In [ ]:
#simulator.optimiser.all_nodes['wells'].devices_serial
#sim_result.dfEdgeFlow.unstack("edge").loc[151]
simulator.optimiser.constrN_wells_energybalance["gas","out",0].pprint()
print(pyo.value(simulator.optimiser.varDeviceFlow["wellL1","gas","out",0]))
print(pyo.value(simulator.optimiser.varDeviceFlow["wellL2","gas","out",0]))
print(pyo.value(simulator.optimiser.varEdgeFlow["cL2g",0]))
print(pyo.value(simulator.optimiser.varEdgeLoss21["cL2g",0]))

In [ ]:
print("CHECKING edge pressure drop on selected edges:")
optimiser = simulator.optimiser
print('w1: {:.5g} MPa'.format(optimiser.all_networks["water"].compute_edge_pressuredrop(optimiser.all_edges['w1'],Q=1.15,p1=0.7,linear=True)))
print('w3: {:.5g} MPa'.format(optimiser.all_networks["water"].compute_edge_pressuredrop(optimiser.all_edges['w3'],Q=1.3/15,p1=7,linear=False)))
print('o2: {:.5g} MPa'.format(optimiser.all_networks["oil"].compute_edge_pressuredrop(optimiser.all_edges['o2'],Q=0.1,p1=5,linear=True)))
print('g2: {:.5g} MPa'.format(optimiser.all_networks["gas"].compute_edge_pressuredrop(optimiser.all_edges['g2'],Q=48,p1=10,linear=True)))

In [ ]:
#mc.checkEdgePressureDrop(timestep=1,var="outer")

In [ ]:
plots.plotter='matplotlib'
fig2=plots.plot_sum_power_mix(sim_result,simulator.optimiser,carrier="el",filename=None,devs_shareload=[])
#fig2.update_layout(autosize=False,width=800,height=500,margin=dict(l=0,r=0,t=0,b=0))
#fig2.write_image("{}/elpowermix_{}.png".format(outpath,case))
fig2.set_size_inches(6,4)
plt.suptitle("")
plt.savefig("result/poweroutput_{}x.pdf".format(case),dpi=300,bbox_inches="tight")

In [ ]:
mean_el_demand = sim_result.dfDeviceFlow.unstack('carrier')['el'].unstack('terminal')['in'].dropna().unstack().T.mean()
mean_heat_demand = sim_result.dfDeviceFlow.unstack('carrier')['heat'].unstack('terminal')['in'].dropna().unstack().T.mean()
energydemand = pd.concat({'el':mean_el_demand,'heat':mean_heat_demand},axis=1)
energydemand.drop('heatdump',inplace=True)
energydemand.plot.barh(title="Mean power demand (MW)")
energydemand

In [ ]:
energydemand.sum()

In [ ]:
mean_flows=sim_result.dfDeviceFlow.unstack(["carrier","device","terminal"]).mean().unstack(["carrier","terminal"])
print("Mean flows:\n",mean_flows.sum())
print("Mean flow per device:")
mean_flows

In [ ]:
mean_flows.loc[["Gen1","Gen2","Gen3"]].sum()

In [ ]:
well_inout=sim_result.dfDeviceFlow.unstack('device')[['wellL1','wellL2']].sum(axis=1).unstack('time').mean(axis=1).unstack('terminal')
print("Net gas production = {:.5g} Sm3/s".format((well_inout['out']-well_inout['in']).sum()))
print("Gas circulated =     {:.5g} Sm3/s".format(well_inout['in'].sum()))

In [ ]:
plots.plotter="plotly"
gts = [d for d,d_obj in simulator.optimiser.all_devices.items() if d_obj.dev_data.model in ["gasturbine","source_el"]]
fig=plots.plot_device_profile(sim_result,simulator.optimiser,devs=gts,filename=None,include_on_off=True, include_prep=False,devs_shareload=None)
fig.update_layout(autosize=False,width=700,height=300,margin=dict(l=0,r=0,t=30,b=0),template="plotly_white")
fig.show()
#300 dpi and 4 inch: 1200 pixels
scale=300/96 # ratio print vs screen dpi
#fig.write_image(outpath+"powersupply.pdf",width=700,height=300,scale=scale)

In [ ]:
dfStartopt = pd.DataFrame()
inst=simulator.optimiser
for t in inst.setHorizon: 
    dfStartopt.loc[t,'start']=inst.varDeviceStarting['Gen2',t].value
    dfStartopt.loc[t,'prep']=inst.varDeviceIsPrep['Gen2',t].value
    dfStartopt.loc[t,'on']=inst.varDeviceIsOn['Gen2',t].value
    dfStartopt.loc[t,'stop']=inst.varDeviceStopping['Gen2',t].value
dfStartopt.head()

In [ ]:
fig=plots.plot_device_profile(sim_result,simulator.optimiser,devs=['wind'],filename=None,include_forecasts=True)
fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=0,b=0))

In [ ]:
df=sim_result.dfDeviceFlow.unstack([0,1,2])[[('ex_g','gas','in'),('ex_o','oil','in')]]
df.columns=['gas','oil']
df=df.reset_index()
df["gas"]=df["gas"]/1000 # convert to oil equivalents
df = df.melt(var_name="Product",value_name="Volume rate (Sm3oe/s)",id_vars=('time'))
px.line(df,x="time",y="Volume rate (Sm3oe/s)",color="Product",title="Oil/gas export volumes (Sm3oe/s)").show()

plots.plot_export_revenue(sim_result)

In [ ]:
simulator.optimiser.setDevice.pprint()

In [ ]:
plots.plotter="plotly"
if 'battery' in simulator.optimiser.setDevice:
    fig=plots.plot_device_profile(sim_result,'battery',
                                 filename=None)#outpath+"battery_opt.png")
    fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0)).show()
    # oogeso.milp_plot.plotDevicePowerLastOptimisation1(mc,device='battery', filename=None)#outpath+"lastopt_battery.png")

In [ ]:
plots.plotter="plotly"
fig=plots.plot_CO2_rate_per_device(sim_result,simulator.optimiser,reverse_legend=True,filename=outpath+"co2rate_opt.png",device_shareload=gts)
fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=0,b=0))

In [ ]:
fig=plots.plot_CO2_intensity(sim_result,filename=outpath+"co2intensity_opt.png")
fig.update_layout(autosize=False,width=700,height=300,margin=dict(l=0,r=0,t=0,b=0))

In [ ]:
plots.plotter="matplotlib"
fig=plots.plot_reserve(sim_result,simulator.optimiser,dynamic_margin=True, include_sum=False)
plt.legend(loc="upper left",bbox_to_anchor=(1,1),frameon=False)
fig.set_size_inches(5.92,1.5)
plt.savefig("result/reserve_{}x.pdf".format(case),dpi=300,bbox_inches="tight")
#fig.update_layout(autosize=False,width=700,height=200,margin=dict(l=0,r=0,t=30,b=0),template="plotly_white")
#fig.update_layout(legend_traceorder="reversed")
#fig.show()
#300 dpi and 4 inch: 1200 pixels
#scale=300/96 # ratio print vs screen dpi
#fig.write_image(outpath+"reserve.pdf",width=700,height=200,scale=scale)

In [ ]:
# This plot shows per device its output (dotted line) and the available online backup, iel reserve 
# by _other_ devices (solid line). MARGIN(t) = min_devices(backup(t) - output(t))
#plots.plotElBackup(mc,showMargin=True,returnMargin=False)

In [ ]:
fig=plots.plot_device_power_energy(sim_result,simulator.optimiser,"h2storage")
fig.update_layout(autosize=False,width=800,height=250,margin=dict(l=0,r=0,t=30,b=0),template="plotly_white")
fig.update_yaxes(tick0=0, dtick=1,secondary_y=True,range=[0,5.5])
fig.update_yaxes(tick0=0, dtick=100e3,secondary_y=False,range=[0,550e3])
fig.show()
fig.write_image(outpath+"h2storage_{}.pdf".format(case))